# To start

In [ ]:
import PIL
from PIL import Image

import numpy as np
from numpy import asarray

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim
from torch.nn.modules.batchnorm import _BatchNorm

from torchvision import transforms

import random
from torchvision.models.feature_extraction import create_feature_extractor
import torchvision.models as models
import math
from torch import Tensor
from torch.nn import functional as F
from torchvision.transforms.functional import InterpolationMode as IMode

import math
import os
import random
import time

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms

from collections import OrderedDict
from tqdm import tqdm
from collections import OrderedDict

In [ ]:
import torch
print("GPU 사용 가능 여부:", torch.cuda.is_available())
print("사용 중인 GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "GPU가 사용 중이 아닙니다.")

GPU 사용 가능 여부: True
사용 중인 GPU 이름: Tesla T4


In [ ]:
# For when using google colab

from google.colab import drive
drive.mount('/content/drive')

drive_root='/content/drive/MyDrive/MAGNet' # Put your own path
print(drive_root)

Mounted at /content/drive
/content/drive/MyDrive/sryeo


# Model

## Discriminator

In [ ]:
# from torch import nn as nn
# from torch.nn import functional as F
# from torch.nn.utils import spectral_norm

# from basicsr.utils.registry import ARCH_REGISTRY

class VGGStyleDiscriminator(nn.Module):
    """VGG style discriminator with input size 128 x 128 or 256 x 256.

    It is used to train SRGAN, ESRGAN, and VideoGAN.

    Args:
        num_in_ch (int): Channel number of inputs. Default: 3.
        num_feat (int): Channel number of base intermediate features.Default: 64.
    """

    def __init__(self, num_in_ch, num_feat, input_size=128):
        super(VGGStyleDiscriminator, self).__init__()
        self.input_size = input_size
        assert self.input_size == 128 or self.input_size == 256, (
            f'input size must be 128 or 256, but received {input_size}')

        self.conv0_0 = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1, bias=True)
        self.conv0_1 = nn.Conv2d(num_feat, num_feat, 4, 2, 1, bias=False)
        self.bn0_1 = nn.BatchNorm2d(num_feat, affine=True)

        self.conv1_0 = nn.Conv2d(num_feat, num_feat * 2, 3, 1, 1, bias=False)
        self.bn1_0 = nn.BatchNorm2d(num_feat * 2, affine=True)
        self.conv1_1 = nn.Conv2d(num_feat * 2, num_feat * 2, 4, 2, 1, bias=False)
        self.bn1_1 = nn.BatchNorm2d(num_feat * 2, affine=True)

        self.conv2_0 = nn.Conv2d(num_feat * 2, num_feat * 4, 3, 1, 1, bias=False)
        self.bn2_0 = nn.BatchNorm2d(num_feat * 4, affine=True)
        self.conv2_1 = nn.Conv2d(num_feat * 4, num_feat * 4, 4, 2, 1, bias=False)
        self.bn2_1 = nn.BatchNorm2d(num_feat * 4, affine=True)

        self.conv3_0 = nn.Conv2d(num_feat * 4, num_feat * 8, 3, 1, 1, bias=False)
        self.bn3_0 = nn.BatchNorm2d(num_feat * 8, affine=True)
        self.conv3_1 = nn.Conv2d(num_feat * 8, num_feat * 8, 4, 2, 1, bias=False)
        self.bn3_1 = nn.BatchNorm2d(num_feat * 8, affine=True)

        self.conv4_0 = nn.Conv2d(num_feat * 8, num_feat * 8, 3, 1, 1, bias=False)
        self.bn4_0 = nn.BatchNorm2d(num_feat * 8, affine=True)
        self.conv4_1 = nn.Conv2d(num_feat * 8, num_feat * 8, 4, 2, 1, bias=False)
        self.bn4_1 = nn.BatchNorm2d(num_feat * 8, affine=True)

        if self.input_size == 256:
            self.conv5_0 = nn.Conv2d(num_feat * 8, num_feat * 8, 3, 1, 1, bias=False)
            self.bn5_0 = nn.BatchNorm2d(num_feat * 8, affine=True)
            self.conv5_1 = nn.Conv2d(num_feat * 8, num_feat * 8, 4, 2, 1, bias=False)
            self.bn5_1 = nn.BatchNorm2d(num_feat * 8, affine=True)

        self.linear1 = nn.Linear(num_feat * 8 * 4 * 4, 80)
        self.linear2 = nn.Linear(80, 1)

        # activation function
        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        assert x.size(2) == self.input_size, (f'Input size must be identical to input_size, but received {x.size()}.')

        feat = self.lrelu(self.conv0_0(x))
        feat = self.lrelu(self.bn0_1(self.conv0_1(feat)))  # output spatial size: /2

        feat = self.lrelu(self.bn1_0(self.conv1_0(feat)))
        feat = self.lrelu(self.bn1_1(self.conv1_1(feat)))  # output spatial size: /4

        feat = self.lrelu(self.bn2_0(self.conv2_0(feat)))
        feat = self.lrelu(self.bn2_1(self.conv2_1(feat)))  # output spatial size: /8

        feat = self.lrelu(self.bn3_0(self.conv3_0(feat)))
        feat = self.lrelu(self.bn3_1(self.conv3_1(feat)))  # output spatial size: /16

        feat = self.lrelu(self.bn4_0(self.conv4_0(feat)))
        feat = self.lrelu(self.bn4_1(self.conv4_1(feat)))  # output spatial size: /32

        if self.input_size == 256:
            feat = self.lrelu(self.bn5_0(self.conv5_0(feat)))
            feat = self.lrelu(self.bn5_1(self.conv5_1(feat)))  # output spatial size: / 64

        # spatial size: (4, 4)
        feat = feat.view(feat.size(0), -1)
        feat = self.lrelu(self.linear1(feat))
        out = self.linear2(feat)
        return out

### GANLoss

In [ ]:
class GANLoss(nn.Module):
    """Define GAN loss.

    Args:
        gan_type (str): Support 'vanilla', 'lsgan', 'wgan', 'hinge'.
        real_label_val (float): The value for real label. Default: 1.0.
        fake_label_val (float): The value for fake label. Default: 0.0.
        loss_weight (float): Loss weight. Default: 1.0.
            Note that loss_weight is only for generators; and it is always 1.0
            for discriminators.
    """

    def __init__(self, gan_type, real_label_val=1.0, fake_label_val=0.0, loss_weight=1.0):
        super(GANLoss, self).__init__()
        self.gan_type = gan_type
        self.loss_weight = loss_weight
        self.real_label_val = real_label_val
        self.fake_label_val = fake_label_val

        if self.gan_type == 'vanilla':
            self.loss = nn.BCEWithLogitsLoss()
        elif self.gan_type == 'lsgan':
            self.loss = nn.MSELoss()
        elif self.gan_type == 'wgan':
            self.loss = self._wgan_loss
        elif self.gan_type == 'wgan_softplus':
            self.loss = self._wgan_softplus_loss
        elif self.gan_type == 'hinge':
            self.loss = nn.ReLU()
        else:
            raise NotImplementedError(f'GAN type {self.gan_type} is not implemented.')

    def _wgan_loss(self, input, target):
        """wgan loss.

        Args:
            input (Tensor): Input tensor.
            target (bool): Target label.

        Returns:
            Tensor: wgan loss.
        """
        return -input.mean() if target else input.mean()

    def _wgan_softplus_loss(self, input, target):
        """wgan loss with soft plus. softplus is a smooth approximation to the
        ReLU function.

        In StyleGAN2, it is called:
            Logistic loss for discriminator;
            Non-saturating loss for generator.

        Args:
            input (Tensor): Input tensor.
            target (bool): Target label.

        Returns:
            Tensor: wgan loss.
        """
        return F.softplus(-input).mean() if target else F.softplus(input).mean()

    def get_target_label(self, input, target_is_real):
        """Get target label.

        Args:
            input (Tensor): Input tensor.
            target_is_real (bool): Whether the target is real or fake.

        Returns:
            (bool | Tensor): Target tensor. Return bool for wgan, otherwise,
                return Tensor.
        """

        if self.gan_type in ['wgan', 'wgan_softplus']:
            return target_is_real
        target_val = (self.real_label_val if target_is_real else self.fake_label_val)
        return input.new_ones(input.size()) * target_val

    def forward(self, input, target_is_real, is_disc=False):
        """
        Args:
            input (Tensor): The input for the loss module, i.e., the network
                prediction.
            target_is_real (bool): Whether the targe is real or fake.
            is_disc (bool): Whether the loss for discriminators or not.
                Default: False.

        Returns:
            Tensor: GAN loss value.
        """
        target_label = self.get_target_label(input, target_is_real)
        if self.gan_type == 'hinge':
            if is_disc:  # for discriminators in hinge-gan
                input = -input if target_is_real else input
                loss = self.loss(1 + input).mean()
            else:  # for generators in hinge-gan
                loss = -input.mean()
        else:  # other gan types
            loss = self.loss(input, target_label)

        # loss_weight is always 1.0 for discriminators
        return loss if is_disc else loss * self.loss_weight

### VGG Extract Feature

In [ ]:
# import os
# import torch
# from collections import OrderedDict
# from torch import nn as nn

# from basicsr.utils.registry import ARCH_REGISTRY

VGG_PRETRAIN_PATH = 'experiments/pretrained_models/vgg19-dcbb9e9d.pth'
NAMES = {
    'vgg19': [
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1', 'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'conv3_4', 'relu3_4', 'pool3', 'conv4_1',
        'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3', 'relu4_3', 'conv4_4', 'relu4_4', 'pool4', 'conv5_1', 'relu5_1',
        'conv5_2', 'relu5_2', 'conv5_3', 'relu5_3', 'conv5_4', 'relu5_4', 'pool5'
    ]
}

def insert_bn(names):
    """Insert bn layer after each conv.

    Args:
        names (list): The list of layer names.

    Returns:
        list: The list of layer names with bn layers.
    """
    names_bn = []
    for name in names:
        names_bn.append(name)
        if 'conv' in name:
            position = name.replace('conv', '')
            names_bn.append('bn' + position)
    return names_bn


class VGGFeatureExtractor(nn.Module):
    """VGG network for feature extraction.

    In this implementation, we allow users to choose whether use normalization
    in the input feature and the type of vgg network. Note that the pretrained
    path must fit the vgg type.

    Args:
        layer_name_list (list[str]): Forward function returns the corresponding
            features according to the layer_name_list.
            Example: {'relu1_1', 'relu2_1', 'relu3_1'}.
        vgg_type (str): Set the type of vgg network. Default: 'vgg19'.
        use_input_norm (bool): If True, normalize the input image. Importantly,
            the input feature must in the range [0, 1]. Default: True.
        range_norm (bool): If True, norm images with range [-1, 1] to [0, 1].
            Default: False.
        requires_grad (bool): If true, the parameters of VGG network will be
            optimized. Default: False.
        remove_pooling (bool): If true, the max pooling operations in VGG net
            will be removed. Default: False.
        pooling_stride (int): The stride of max pooling operation. Default: 2.
    """

    def __init__(self,
                 layer_name_list,
                 vgg_type='vgg19',
                 use_input_norm=True,
                 range_norm=False,
                 requires_grad=False,
                 remove_pooling=False,
                 pooling_stride=2):
        super(VGGFeatureExtractor, self).__init__()

        self.layer_name_list = layer_name_list
        self.use_input_norm = use_input_norm
        self.range_norm = range_norm

        self.names = NAMES[vgg_type.replace('_bn', '')]
        if 'bn' in vgg_type:
            self.names = insert_bn(self.names)

        # only borrow layers that will be used to avoid unused params
        max_idx = 0
        for v in layer_name_list:
            idx = self.names.index(v)
            if idx > max_idx:
                max_idx = idx

        if os.path.exists(VGG_PRETRAIN_PATH):
            vgg_net = getattr(models.vgg, vgg_type)(pretrained=False)
            state_dict = torch.load(VGG_PRETRAIN_PATH, map_location=lambda storage, loc: storage)
            vgg_net.load_state_dict(state_dict)
        else:
            vgg_net = getattr(models.vgg, vgg_type)(pretrained=True)

        features = vgg_net.features[:max_idx + 1]

        modified_net = OrderedDict()
        for k, v in zip(self.names, features):
            if 'pool' in k:
                # if remove_pooling is true, pooling operation will be removed
                if remove_pooling:
                    continue
                else:
                    # in some cases, we may want to change the default stride
                    modified_net[k] = nn.MaxPool2d(kernel_size=2, stride=pooling_stride)
            else:
                modified_net[k] = v

        self.vgg_net = nn.Sequential(modified_net)

        if not requires_grad:
            self.vgg_net.eval()
            for param in self.parameters():
                param.requires_grad = False
        else:
            self.vgg_net.train()
            for param in self.parameters():
                param.requires_grad = True

        if self.use_input_norm:
            # the mean is for image with range [0, 1]
            self.register_buffer('mean', torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
            # the std is for image with range [0, 1]
            self.register_buffer('std', torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))

    def forward(self, x):
        """Forward function.

        Args:
            x (Tensor): Input tensor with shape (n, c, h, w).

        Returns:
            Tensor: Forward results.
        """
        if self.range_norm:
            x = (x + 1) / 2
        if self.use_input_norm:
            x = (x - self.mean) / self.std

        output = {}
        for key, layer in self.vgg_net._modules.items():
            x = layer(x)
            if key in self.layer_name_list:
                output[key] = x.clone()

        return output

### Compaact VGG Net

In [ ]:
class SRVGGNetCompact(nn.Module):
    """A compact VGG-style network structure for super-resolution.

    It is a compact network structure, which performs upsampling in the last layer and no convolution is
    conducted on the HR feature space.

    Args:
        num_in_ch (int): Channel number of inputs. Default: 3.
        num_out_ch (int): Channel number of outputs. Default: 3.
        num_feat (int): Channel number of intermediate features. Default: 64.
        num_conv (int): Number of convolution layers in the body network. Default: 16.
        upscale (int): Upsampling factor. Default: 4.
        act_type (str): Activation type, options: 'relu', 'prelu', 'leakyrelu'. Default: prelu.
    """

    def __init__(self, num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=16, upscale=4, act_type='prelu'):
        super(SRVGGNetCompact, self).__init__()
        self.num_in_ch = num_in_ch
        self.num_out_ch = num_out_ch
        self.num_feat = num_feat
        self.num_conv = num_conv
        self.upscale = upscale
        self.act_type = act_type

        self.body = nn.ModuleList()
        # the first conv
        self.body.append(nn.Conv2d(num_in_ch, num_feat, 3, 1, 1))
        # the first activation
        if act_type == 'relu':
            activation = nn.ReLU(inplace=True)
        elif act_type == 'prelu':
            activation = nn.PReLU(num_parameters=num_feat)
        elif act_type == 'leakyrelu':
            activation = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.body.append(activation)

        # the body structure
        for _ in range(num_conv):
            self.body.append(nn.Conv2d(num_feat, num_feat, 3, 1, 1))
            # activation
            if act_type == 'relu':
                activation = nn.ReLU(inplace=True)
            elif act_type == 'prelu':
                activation = nn.PReLU(num_parameters=num_feat)
            elif act_type == 'leakyrelu':
                activation = nn.LeakyReLU(negative_slope=0.1, inplace=True)
            self.body.append(activation)

        # the last conv
        self.body.append(nn.Conv2d(num_feat, num_out_ch * upscale * upscale, 3, 1, 1))
        # upsample
        self.upsampler = nn.PixelShuffle(upscale)

    def forward(self, x):
        out = x
        for i in range(0, len(self.body)):
            out = self.body[i](out)

        out = self.upsampler(out)
        # add the nearest upsampled image, so that the network learns the residual
        base = F.interpolate(x, scale_factor=self.upscale, mode='nearest')
        out += base
        return out

### Perceptual Loss

In [ ]:
class PerceptualLoss(nn.Module):
    """Perceptual loss with commonly used style loss.

    Args:
        layer_weights (dict): The weight for each layer of vgg feature.
            Here is an example: {'conv5_4': 1.}, which means the conv5_4
            feature layer (before relu5_4) will be extracted with weight
            1.0 in calculating losses.
        vgg_type (str): The type of vgg network used as feature extractor.
            Default: 'vgg19'.
        use_input_norm (bool):  If True, normalize the input image in vgg.
            Default: True.
        range_norm (bool): If True, norm images with range [-1, 1] to [0, 1].
            Default: False.
        perceptual_weight (float): If `perceptual_weight > 0`, the perceptual
            loss will be calculated and the loss will multiplied by the
            weight. Default: 1.0.
        style_weight (float): If `style_weight > 0`, the style loss will be
            calculated and the loss will multiplied by the weight.
            Default: 0.
        criterion (str): Criterion used for perceptual loss. Default: 'l1'.
    """

    def __init__(self,
                 layer_weights,
                 vgg_type='vgg19',
                 use_input_norm=True,
                 range_norm=False,
                 perceptual_weight=1.0,
                 style_weight=0.,
                 criterion='l1'):
        super(PerceptualLoss, self).__init__()
        self.perceptual_weight = perceptual_weight
        self.style_weight = style_weight
        self.layer_weights = layer_weights
        self.vgg = VGGFeatureExtractor(
            layer_name_list=list(layer_weights.keys()),
            vgg_type=vgg_type,
            use_input_norm=use_input_norm,
            range_norm=range_norm)

        self.criterion_type = criterion
        if self.criterion_type == 'l1':
            self.criterion = torch.nn.L1Loss()
        elif self.criterion_type == 'l2':
            self.criterion = torch.nn.MSELoss()
        elif self.criterion_type == 'fro':
            self.criterion = None
        else:
            raise NotImplementedError(f'{criterion} criterion has not been supported.')

    def forward(self, x, gt):
        """Forward function.

        Args:
            x (Tensor): Input tensor with shape (n, c, h, w).
            gt (Tensor): Ground-truth tensor with shape (n, c, h, w).

        Returns:
            Tensor: Forward results.
        """
        # extract vgg features
        x_features = self.vgg(x)
        gt_features = self.vgg(gt.detach())

        # calculate perceptual loss
        if self.perceptual_weight > 0:
            percep_loss = 0
            for k in x_features.keys():
                if self.criterion_type == 'fro':
                    percep_loss += torch.norm(x_features[k] - gt_features[k], p='fro') * self.layer_weights[k]
                else:
                    percep_loss += self.criterion(x_features[k], gt_features[k]) * self.layer_weights[k]
            percep_loss *= self.perceptual_weight
        else:
            percep_loss = None

        # calculate style loss
        if self.style_weight > 0:
            style_loss = 0
            for k in x_features.keys():
                if self.criterion_type == 'fro':
                    style_loss += torch.norm(
                        self._gram_mat(x_features[k]) - self._gram_mat(gt_features[k]), p='fro') * self.layer_weights[k]
                else:
                    style_loss += self.criterion(self._gram_mat(x_features[k]), self._gram_mat(
                        gt_features[k])) * self.layer_weights[k]
            style_loss *= self.style_weight
        else:
            style_loss = None

        return percep_loss, style_loss

    def _gram_mat(self, x):
        """Calculate Gram matrix.

        Args:
            x (torch.Tensor): Tensor with shape of (n, c, h, w).

        Returns:
            torch.Tensor: Gram matrix.
        """
        n, c, h, w = x.size()
        features = x.view(n, c, w * h)
        features_t = features.transpose(1, 2)
        gram = features.bmm(features_t) / (c * h * w)
        return gram

## Generator Model

### Meta-sr module

In [ ]:
def repeat(x: Tensor, upscale_factor: int) -> Tensor:
    batch_size, channels, height, width = x.size()
    out = x.view(batch_size, channels, height, 1, width, 1)

    upscale_factor = math.ceil(upscale_factor)
    out = torch.cat([out] * upscale_factor, 3)
    out = torch.cat([out] * upscale_factor, 5).permute(0, 3, 5, 1, 2, 4)

    out = out.contiguous().view(-1, channels, height, width)

    return out

def weight_prediction_matrix_from_lr(lr_image_height: int, lr_image_width: int, upscale_factor: float):
    """Weight prediction matrix from LR to SR"""
    sr_image_height, sr_image_width = int(upscale_factor * lr_image_height), int(upscale_factor * lr_image_width)

    # Calculate the input matrix by calculating the offset
    upscale_factor_int = int(math.ceil(upscale_factor))

    height_offset = torch.ones(lr_image_height, upscale_factor_int, 1)
    width_offset = torch.ones(1, lr_image_width, upscale_factor_int)

    height_mask = torch.zeros(lr_image_height, upscale_factor_int, 1)
    width_mask = torch.zeros(1, lr_image_width, upscale_factor_int)

    upscale_factor_matrix = torch.zeros(1, 1)
    upscale_factor_matrix[0, 0] = 1.0 / upscale_factor
    # (lr_image_width * lr_image_height * up_scale_factor ** 2, 1)
    upscale_factor_matrix = torch.cat([upscale_factor_matrix] * (lr_image_height * lr_image_width * (upscale_factor_int ** 2)), 0)

    # Calculate projected coordinate offsets
    height_project_coord_offset = torch.arange(0, sr_image_height, 1).float().mul(1.0 / upscale_factor)
    width_project_coord_offset = torch.arange(0, sr_image_width, 1).float().mul(1.0 / upscale_factor)

    floor_height_project_coord_offset = torch.floor(height_project_coord_offset)
    floor_width_project_coord_offset = torch.floor(width_project_coord_offset)

    height_coord_offset = height_project_coord_offset - floor_height_project_coord_offset
    width_coord_offset = width_project_coord_offset - floor_width_project_coord_offset

    floor_height_project_coord_offset = floor_height_project_coord_offset.int()
    floor_width_project_coord_offset = floor_width_project_coord_offset.int()

    # Calculate the LR coordinate offset
    flag = 0
    number = 0
    for i in range(sr_image_height):
        if floor_height_project_coord_offset[i] == number:
            height_offset[floor_height_project_coord_offset[i], flag, 0] = height_coord_offset[i]
            height_mask[floor_height_project_coord_offset[i], flag, 0] = 1
            flag += 1
        else:
            height_offset[floor_height_project_coord_offset[i], 0, 0] = height_coord_offset[i]
            height_mask[floor_height_project_coord_offset[i], 0, 0] = 1
            number += 1
            flag = 1

    flag = 0
    number = 0
    for i in range(sr_image_width):
        if floor_width_project_coord_offset[i] == number:
            width_offset[0, floor_width_project_coord_offset[i], flag] = width_coord_offset[i]
            width_mask[0, floor_width_project_coord_offset[i], flag] = 1
            flag += 1
        else:
            width_offset[0, floor_width_project_coord_offset[i], 0] = width_coord_offset[i]
            width_mask[0, floor_width_project_coord_offset[i], 0] = 1
            number += 1
            flag = 1

    height_coord_offset = torch.cat([height_offset] * (upscale_factor_int * lr_image_width), 2).view(-1, upscale_factor_int * lr_image_width, 1)
    width_coord_offset = torch.cat([width_offset] * (upscale_factor_int * lr_image_height), 0).view(-1, upscale_factor_int * lr_image_width, 1)

    height_mask = torch.cat([height_mask] * (upscale_factor_int * lr_image_width), 2).view(-1, upscale_factor_int * lr_image_width, 1)
    width_mask = torch.cat([width_mask] * (upscale_factor_int * lr_image_height), 0).view(-1, upscale_factor_int * lr_image_width, 1)

    pos_matrix = torch.cat((height_coord_offset, width_coord_offset), 2)
    mask_matrix = torch.sum(torch.cat((height_mask, width_mask), 2), 2).view(upscale_factor_int * lr_image_height, upscale_factor_int * lr_image_width)
    mask_matrix = mask_matrix.eq(2)
    pos_matrix = pos_matrix.contiguous().view(1, -1, 2)

    pos_matrix = torch.cat((upscale_factor_matrix.view(1, -1, 1), pos_matrix), 2)

    return pos_matrix, mask_matrix

class _PosToWeight(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(_PosToWeight, self).__init__()
        self.pos_to_weight = nn.Sequential(
            nn.Linear(3, 256),
            nn.ReLU(True),
            nn.Linear(256, 256),
            nn.ReLU(True),
            nn.Linear(256, 3 * 3 * in_channels * out_channels)
        )

    def forward(self, x: Tensor) -> Tensor:
        out = self.pos_to_weight(x)

        return out

### RRDB

In [ ]:
@torch.no_grad()
def default_init_weights(module_list, scale=1, bias_fill=0, **kwargs):
    """Initialize network weights.

    Args:
        module_list (list[nn.Module] | nn.Module): Modules to be initialized.
        scale (float): Scale initialized weights, especially for residual
            blocks. Default: 1.
        bias_fill (float): The value to fill bias. Default: 0
        kwargs (dict): Other arguments for initialization function.
    """
    if not isinstance(module_list, list):
        module_list = [module_list]
    for module in module_list:
        for m in module.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, **kwargs)
                m.weight.data *= scale
                if m.bias is not None:
                    m.bias.data.fill_(bias_fill)

class ResidualDenseBlock(nn.Module):
    """Residual Dense Block.

    Used in RRDB block in ESRGAN.

    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for each growth.
    """

    def __init__(self, num_feat=64, num_grow_ch=32):
        super(ResidualDenseBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1)
        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1)
        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch, num_feat, 3, 1, 1)

        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        # initialization
        default_init_weights([self.conv1, self.conv2, self.conv3, self.conv4, self.conv5], 0.1)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        # Empirically, we use 0.2 to scale the residual for better performance
        return x5 * 0.2 + x

class RRDB(nn.Module):
    """Residual in Residual Dense Block.

    Used in RRDB-Net in ESRGAN.

    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for each growth.
    """

    def __init__(self, num_feat, num_grow_ch=32):
        super(RRDB, self).__init__()
        self.rdb1 = ResidualDenseBlock(num_feat, num_grow_ch)
        self.rdb2 = ResidualDenseBlock(num_feat, num_grow_ch)
        self.rdb3 = ResidualDenseBlock(num_feat, num_grow_ch)

    def forward(self, x):
        out = self.rdb1(x)
        out = self.rdb2(out)
        out = self.rdb3(out)
        # Empirically, we use 0.2 to scale the residual for better performance
        return out * 0.2 + x

def make_layer(basic_block, num_basic_block, **kwarg):
    """Make layers by stacking the same blocks.

    Args:
        basic_block (nn.module): nn.module class for basic block.
        num_basic_block (int): number of blocks.

    Returns:
        nn.Sequential: Stacked blocks in nn.Sequential.
    """
    layers = []
    for _ in range(num_basic_block):
        layers.append(basic_block(**kwarg))
    return nn.Sequential(*layers)

### Main model

In [ ]:
class RRDBNet(nn.Module):
    """Networks consisting of Residual in Residual Dense Block, which is used
    in ESRGAN.

    ESRGAN: Enhanced Super-Resolution Generative Adversarial Networks.

    We extend ESRGAN for scale x2 and scale x1.
    Note: This is one option for scale 1, scale 2 in RRDBNet.
    We first employ the pixel-unshuffle (an inverse operation of pixelshuffle to reduce the spatial size
    and enlarge the channel size before feeding inputs into the main ESRGAN architecture.

    Args:
        num_in_ch (int): Channel number of inputs.
        num_out_ch (int): Channel number of outputs.
        num_feat (int): Channel number of intermediate features.
            Default: 64
        num_block (int): Block number in the trunk network. Defaults: 23
        num_grow_ch (int): Channels for each growth. Default: 32.
    """

    def __init__(self, num_in_ch=3, num_feat=64, num_block=23, num_grow_ch=32):
        super(RRDBNet, self).__init__()

        self.conv_last1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv_last2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv_last3 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv_last1.weight, a=0, mode='fan_in')
        nn.init.constant_(self.conv_last1.bias, 0)
        nn.init.kaiming_normal_(self.conv_last2.weight, a=0, mode='fan_in')
        nn.init.constant_(self.conv_last2.bias, 0)
        nn.init.kaiming_normal_(self.conv_last3.weight, a=0, mode='fan_in')
        nn.init.constant_(self.conv_last3.bias, 0)

        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1)
        self.body = make_layer(RRDB, num_block, num_feat=num_feat, num_grow_ch=num_grow_ch)
        self.conv_body = nn.Conv2d(num_feat, num_feat, 3, 1, 1)

        self.pos_to_weight = _PosToWeight(64, 3)

        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    # Meta-sr
    def meta_sr(self, out, pos_matrix, upscale_factor):
        upscale_factor_int = int(math.ceil(upscale_factor))

        pos_matrix = pos_matrix.view(pos_matrix.size(1), -1)
        local_weight = self.pos_to_weight(pos_matrix)

        repeat_out = repeat(out, upscale_factor_int)
        cols = nn.functional.unfold(repeat_out, (3, 3), padding=1)

        cols = cols.contiguous().view(cols.size(0) // (upscale_factor_int ** 2),
                                      upscale_factor_int ** 2,
                                      cols.size(1),
                                      cols.size(2), 1).permute(0, 1, 3, 4, 2).contiguous()

        local_weight = local_weight.contiguous().view(out.size(2),
                                                      upscale_factor_int,
                                                      out.size(3),
                                                      upscale_factor_int,
                                                      -1,
                                                      3).permute(1, 3, 0, 2, 4, 5).contiguous()
        local_weight = local_weight.contiguous().view(upscale_factor_int ** 2,
                                                      out.size(2) * out.size(3),
                                                      -1,
                                                      3)

        outs = torch.matmul(cols, local_weight).permute(0, 1, 4, 2, 3)
        outs = outs.contiguous().view(out.size(0),
                                      upscale_factor_int,
                                      upscale_factor_int,
                                      3,
                                      out.size(2),
                                      out.size(3)).permute(0, 3, 4, 1, 5, 2)
        outs = outs.contiguous().view(out.size(0),
                                      3,
                                      upscale_factor_int * out.size(2),
                                      upscale_factor_int * out.size(3))

        return outs

    def forward(self, x, pos_matrix, upscale_factor):
        feat = x
        feat = self.conv_first(feat)
        body_feat = self.conv_body(self.body(feat))
        feat = feat + body_feat

        # meta-sr
        out = self.meta_sr(feat, pos_matrix, upscale_factor)
        residual = self.lrelu(self.conv_last1(out))
        residual = self.lrelu(self.conv_last2(residual))
        residual = self.conv_last3(residual)

        out = out + residual
        out = out.clamp_(0.0, 1.0)

        return out

## PSNR

In [ ]:
#Return psnr value between torch.tensor images
def psnr_between_rgb(img1,img2):
    if len(img1.shape) == 4:
        img1 = img1.squeeze(0)
        img2 = img2.squeeze(0)
    y1 = 16. + (64.738 * img1[0, :, :] + 129.057 * img1[1, :, :] + 25.064 * img1[2, :, :]) / 256.
    y2 = 16. + (64.738 * img2[0, :, :] + 129.057 * img2[1, :, :] + 25.064 * img2[2, :, :]) / 256.
    psnr = (10. * torch.log10(1. / torch.mean((y1 - y2) ** 2)))
    return psnr

# Training

In [ ]:
def get_data(data, upscale_factor, lr_image_size, device):
    gt = transforms.RandomCrop([int(upscale_factor * lr_image_size),
                                    int(upscale_factor * lr_image_size)])(data)
    lr = transforms.Resize([lr_image_size, lr_image_size], interpolation=IMode.BICUBIC)(gt)

    gt = gt.to(device, non_blocking=True)
    lr = lr.to(device, non_blocking=True)

    return gt, lr

## Without Discriminator

### Hyper Parameters & Preparation

In [ ]:
##########  MODEL
model_root = os.path.join(drive_root,"model")

######### PARAMETER
model_load = None

learning_rate = 1e-4
training_epochs = 15

batchsize = 4
num_workers = 0
userseed = 123

lr_image_size = 60
min_scale, max_scale = 1.95, 4.05
####################################################################################

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(userseed)

class srDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        super(srDataset, self).__init__()
        self.root_dir = root_dir
        self.files = os.listdir(root_dir)
        #del self.files[100:]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.files[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        image = np.array(image).transpose((2,0,1)).astype(np.float32)/255.

        return image

train_dataset = srDataset(os.path.join(drive_root,'dataset/train'), transform = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5)
]))

train_dataloader = DataLoader(dataset = train_dataset,
                                batch_size = batchsize,
                                shuffle = True,
                                num_workers = num_workers,
                                pin_memory=True,
                                drop_last = True,)

valid_dataset = srDataset(os.path.join(drive_root,'dataset/valid'), transform = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5)
]))

valid_dataloader = DataLoader(dataset = valid_dataset,
                                batch_size = batchsize,
                                shuffle = True,
                                num_workers = num_workers,
                                pin_memory=True,
                                drop_last = True,)

if device == 'cuda':
    torch.cuda.manual_seed_all(userseed)

#models
model = RRDBNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#loading model state
if model_load is not None:
    model.load_state_dict(torch.load(os.path.join(model_root, model_load),weights_only=True))
    print("model loaded: ")

model loaded: 


In [ ]:
### load ESRGAN pretrained model
### Run when not loading other models

weights = torch.load(os.path.join(drive_root, "pretrained", "RRDB_ESRGAN_x4.pth"))

dic = {"conv_first.weight": "conv_first.weight", "conv_first.bias": "conv_first.bias", "trunk_conv.weight": "conv_body.weight", "trunk_conv.bias": "conv_body.bias"}
for i in range(23):
    for j in range(3):
        for k in range(5):
            dic[f"RRDB_trunk.{i}.RDB{j+1}.conv{k+1}.weight"] = f"body.{i}.rdb{j+1}.conv{k+1}.weight"
            dic[f"RRDB_trunk.{i}.RDB{j+1}.conv{k+1}.bias"] = f"body.{i}.rdb{j+1}.conv{k+1}.bias"

layer_weights = {v: weights[k]  for k, v in dic.items()}

model = RRDBNet().to(device)
model.load_state_dict(layer_weights, strict=False)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


<ipython-input-27-323fef29bf68>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(os.path.join(drive_root, "model","RRDB_ESRGAN_x4.pth"))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/sryeo/model/RRDB_ESRGAN_x4.pth'

### Training

In [ ]:
# 학습 시작 시간
import time
from tqdm import tqdm
start_time = time.time()

for epoch in range(training_epochs):
    model.train()
    avg_cost=0
    epoch_start_time = time.time()  # 에포크 시작 시간

    # Freeze all model parameters
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the parameters of the new convolutional layers
    for param in model.conv_last1.parameters():
        param.requires_grad = True
    for param in model.conv_last2.parameters():
        param.requires_grad = True
    for param in model.conv_last3.parameters():
        param.requires_grad = True

    for  batch_idx, data in enumerate(tqdm(train_dataloader)):
        upscale_factor = max(random.uniform(min_scale, max_scale),random.uniform(min_scale, max_scale))


        # According to the requirement in the paper, the lr image size should be 50
        gt, lr = get_data(data, upscale_factor, lr_image_size, device)

        # gt = transforms.RandomCrop([int(upscale_factor * lr_image_size),
        #                             int(upscale_factor * lr_image_size)])(data)
        # lr = transforms.Resize([lr_image_size, lr_image_size], interpolation=IMode.BICUBIC)(gt)

        # gt = gt.to(device, non_blocking=True)
        # lr = lr.to(device, non_blocking=True)

        batch_size, channels, lr_height, lr_width = lr.size()
        _, _, gt_height, gt_width = gt.size()

        # Get the position matrix, mask
        pos_matrix, mask_matrix = weight_prediction_matrix_from_lr(lr_height, lr_width, upscale_factor)
        pos_matrix = pos_matrix.to(device, non_blocking=True)
        mask_matrix = mask_matrix.to(device, non_blocking=True)

        # Initialize the generator gradient
        model.zero_grad(set_to_none=True)

        sr = model(lr, pos_matrix, upscale_factor)
        sr = torch.masked_select(sr, mask_matrix)
        sr = sr.contiguous().view(batch_size, channels, gt_height, gt_width)
        loss = criterion(sr, gt)

        # loss
        avg_cost += loss

        # Backpropagation
        loss.backward()

        # update generator weights
        optimizer.step()

        # 배치 단위로 손실 출력 (1000 배치마다 출력)
        if(batch_idx + 1) % 1000 == 0:
            batch_time = time.time() - epoch_start_time
            print(f"[Batch {batch_idx+1}/{len(train_dataloader)}] [L1 Loss: {loss.item():.4f}] {batch_time:.1f}s")


    # 에포크 완료 후 평균 손실과 경과 시간 출력
    epoch_time = time.time() - epoch_start_time
    avg_cost /= len(train_dataloader)  # 에포크 단위 평균 손실 계산
    print(f"[Epoch {epoch + 1}] cost = {avg_cost:.9f}, Time = {epoch_time:.1f}s")

    # Validation set
    model.eval()
    sum_psnr = 0

    for data in tqdm(valid_dataloader):
        upscale_factor = max(random.uniform(min_scale, max_scale),random.uniform(min_scale, max_scale))

        # According to the requirement in the paper, the lr image size should be 50
        gt = transforms.RandomCrop([int(upscale_factor * lr_image_size),
                                    int(upscale_factor * lr_image_size)])(data)
        lr = transforms.Resize([lr_image_size, lr_image_size], interpolation=IMode.BICUBIC)(gt)

        gt = gt.to(device, non_blocking=True)
        lr = lr.to(device, non_blocking=True)

        # Get the position matrix, mask
        batch_size, channels, lr_height, lr_width = lr.size()
        _, _, gt_height, gt_width = gt.size()
        pos_matrix, mask_matrix = weight_prediction_matrix_from_lr(lr_height, lr_width, upscale_factor)
        pos_matrix = pos_matrix.to(device, non_blocking=True)
        mask_matrix = mask_matrix.to(device, non_blocking=True)

        with torch.no_grad():
            sr = model(lr, pos_matrix, upscale_factor)
            sr = torch.masked_select(sr, mask_matrix)
            sr = sr.contiguous().view(batch_size, channels, gt_height, gt_width)

            sum_psnr += psnr_between_rgb(gt, sr)

    print('PSNR: {:.2f}'.format(sum_psnr/len(valid_dataloader)))

    # 5 epoch 마다 저장
    if (epoch+1) % 5 == 0:
        torch.save(model.state_dict(), os.path.join(model_root,f"g_model_{epoch+1}.pth"))
        print("saved")

print("done")
torch.save(model.state_dict(), os.path.join(model_root,"g_model.pth"))
print("saved")

## With Discriminator

In [ ]:
def zero_pad_to(images, sz):
    _, _, h, w = images.shape
    target_size = int(sz)

    pad_h = (target_size - h) // 2
    pad_w = (target_size - w) // 2

    padding = (pad_w, target_size - w - pad_w, pad_h, target_size - h - pad_h)

    # Apply padding to each image in the batch
    padded_images = nn.functional.pad(images, padding, mode='constant', value=0)
    return padded_images

### Hype Parameters & Preperation

In [ ]:
##########  MODEL
# model_parameters_path = os.path.join(drive_root,"model/model.pth")
# d_model_parameters_path = os.path.join(drive_root,"model/d_model.pth")

model_root = os.path.join(drive_root,"model")

g_model_load = "g_model.pth"
d_model_load = None

######### PARAMETER
# g_from_scratch = False
# d_from_scratch = True
learning_rate = 7e-6
lr2 = 4e-6
lr3 = 7e-6

start_epoch = 0
training_epochs = 100
batchsize = 4
num_workers = 0
userseed = 123

lr_image_size = 60
pad_image_size = 256

min_scale=1.95
max_scale=4.05
# ema_decay = 0.999
####################################################################################

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(userseed)

class srDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        super(srDataset, self).__init__()
        self.root_dir = root_dir
        self.files = os.listdir(root_dir)
        # del self.files[100:]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.files[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        image = np.array(image).transpose((2,0,1)).astype(np.float32)/255.

        return image

train_dataset = srDataset(os.path.join(drive_root,'dataset/train'), transform = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5)
]))

train_dataloader = DataLoader(dataset = train_dataset,
                                batch_size = batchsize,
                                shuffle = True,
                                num_workers = num_workers,
                                pin_memory=True,
                                drop_last = True,)

valid_dataset = srDataset(os.path.join(drive_root,'dataset/valid'), transform = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5)
]))

valid_dataloader = DataLoader(dataset = valid_dataset,
                                batch_size = batchsize,
                                shuffle = True,
                                num_workers = num_workers,
                                pin_memory=True,
                                drop_last = True,)

if device == 'cuda':
    torch.cuda.manual_seed_all(userseed)

#models
g_model = RRDBNet().to(device)
d_model = VGGStyleDiscriminator(3, 64, 256).to(device)


###

# g_optimizer = torch.optim.Adam(
#     list(g_model.conv_last1.parameters()) + list(g_model.conv_last2.parameters()) + list(g_model.conv_last3.parameters()),
#     lr=learning_rate
# )
# Define the parameters explicitly for Group 1
# Define the parameter groups explicitly by layer
group1_params = list(g_model.conv_last1.parameters()) + \
                list(g_model.conv_last2.parameters()) + \
                list(g_model.conv_last3.parameters())

# Use named_parameters to collect all model parameters
group1_names = [id(p) for p in group1_params]  # Use `id` to track parameter memory addresses
group2_params = [p for p in g_model.parameters() if id(p) not in group1_names]

# Define the optimizer
g_optimizer = torch.optim.Adam(
    [
        # Group 1: Specific layers with learning rate `learning_rate`
        {'params': group1_params, 'lr': learning_rate},

        # Group 2: Remaining layers with learning rate `lr2`
        {'params': group2_params, 'lr': lr2}
    ]
)


###
d_optimizer = torch.optim.Adam(d_model.parameters(), lr = lr3)

cri_perceptual = PerceptualLoss({'conv5_4': 1}, 'vgg19', True, False, 1.0, 0, 'l1').to(device)
cri_gan = GANLoss('wgan_softplus', 1, 0, 5e-3).to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#loading model state
# if g_from_scratch is False:
if g_model_load is not None:
    g_model.load_state_dict(torch.load(os.path.join(model_root, g_model_load),weights_only=True))
    print("g_model loaded:", os.path.join(model_root, g_model_load))

# if d_from_scratch is False:
if d_model_load is not None:
    d_model.load_state_dict(torch.load(os.path.join(model_root, d_model_load), weights_only=True))
    print("d_model loaded:", os.path.join(model_root, d_model_load))
else:
    print("d_model automatically loaded")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:10<00:00, 52.9MB/s]


g_model loaded: /content/drive/MyDrive/sryeo/model/g_model_50.pth
d_model loaded: /content/drive/MyDrive/sryeo/model/d_model_50.pth


### Training

In [ ]:
start_time = time.time()


for epoch in range(start_epoch, training_epochs):
    g_model.train()
    d_model.train()



    avg_l_g = 0
    avg_l_d = 0

    for data in tqdm(train_dataloader):
        ### data processing
        #upscale_factor = random.choice(upscale_factors)
        upscale_factor = max (random.uniform(min_scale, max_scale), random.uniform(min_scale, max_scale))

        gt, lr = get_data(data, upscale_factor, lr_image_size, device)

        # gt = transforms.RandomCrop([int(upscale_factor * lr_image_size),
        #                             int(upscale_factor * lr_image_size)])(data)
        # lr = transforms.Resize([lr_image_size, lr_image_size], interpolation=IMode.BICUBIC)(gt)
        # gt = gt.to(device, non_blocking=True)
        # lr = lr.to(device, non_blocking=True)

        batch_size, channels, lr_height, lr_width = lr.size()
        _, _, gt_height, gt_width = gt.size()

        pos_matrix, mask_matrix = weight_prediction_matrix_from_lr(lr_height, lr_width, upscale_factor)
        pos_matrix = pos_matrix.to(device, non_blocking=True)
        mask_matrix = mask_matrix.to(device, non_blocking=True)

        for p in d_model.parameters():
            p.requires_grad = False

        g_optimizer.zero_grad()
        sr = g_model(lr, pos_matrix, upscale_factor)
        sr = torch.masked_select(sr, mask_matrix)
        sr = sr.contiguous().view(batch_size, channels, gt_height, gt_width)

        sr = zero_pad_to(sr, pad_image_size)
        gt = zero_pad_to(gt, pad_image_size)

        l_g_total = 0
        # loss_dict = OrderedDict()

        # perceptual loss
        l_g_percep, l_g_style = cri_perceptual(sr, gt)
        if l_g_percep is not None:
            l_g_total += l_g_percep
            # loss_dict['l_g_percep'] = l_g_percep
        if l_g_style is not None:
            l_g_total += l_g_style
            # loss_dict['l_g_style'] = l_g_style

        # gan loss (relativistic gan)
        real_d_pred = d_model(gt).detach()
        fake_g_pred = d_model(sr)
        l_g_real = cri_gan(real_d_pred - torch.mean(fake_g_pred), False, is_disc=False)
        l_g_fake = cri_gan(fake_g_pred - torch.mean(real_d_pred), True, is_disc=False)
        l_g_gan = (l_g_real + l_g_fake) / 2

        l_g_total += l_g_gan
        # loss_dict['l_g_gan'] = l_g_gan
        avg_l_g += l_g_gan

        l_g_total.backward()
        g_optimizer.step()

        # optimize net_d
        for p in d_model.parameters():
            p.requires_grad = True

        d_optimizer.zero_grad()
        # gan loss (relativistic gan)

        # In order to avoid the error in distributed training:
        # "Error detected in CudnnBatchNormBackward: RuntimeError: one of
        # the variables needed for gradient computation has been modified by
        # an inplace operation",
        # we separate the backwards for real and fake, and also detach the
        # tensor for calculating mean.

        # real
        fake_d_pred = d_model(sr).detach()
        real_d_pred = d_model(gt)
        l_d_real = cri_gan(real_d_pred - torch.mean(fake_d_pred), True, is_disc=True) * 0.5
        l_d_real.backward()
        # fake
        fake_d_pred = d_model(sr.detach())
        l_d_fake = cri_gan(fake_d_pred - torch.mean(real_d_pred.detach()), False, is_disc=True) * 0.5
        l_d_fake.backward()
        d_optimizer.step()

        avg_l_d += l_d_real + l_d_fake

    avg_l_g /= len(train_dataloader)
    avg_l_d /= len(train_dataloader)
    print('[Epoch: {:>4}] avg_gen_loss = {:>.9} avg_dis_loss = {:>.9}'.format(epoch + 1, avg_l_g, avg_l_d))




    if (epoch+1) % 5 == 0:
    #if True:
        torch.save(g_model.state_dict(), os.path.join(model_root, f"g_model_{epoch+1}.pth"))
        torch.save(d_model.state_dict(), os.path.join(model_root, f"d_model_{epoch+1}.pth"))
        print("saved")
        ### eval
        g_model.eval()
        d_model.eval()
        sum_psnr = 0
        f_score = 0
        t_score = 0

        for data in tqdm(valid_dataloader):
            upscale_factor = random.choice(upscale_factors)

            gt, lr = get_data(data, upscale_factor, lr_image_size, device)

            batch_size, channels, lr_height, lr_width = lr.size()
            _, _, gt_height, gt_width = gt.size()

            pos_matrix, mask_matrix = weight_prediction_matrix_from_lr(lr_height, lr_width, upscale_factor)
            pos_matrix = pos_matrix.to(device, non_blocking=True)
            mask_matrix = mask_matrix.to(device, non_blocking=True)

            with torch.no_grad():
                sr = g_model(lr, pos_matrix, upscale_factor)
                sr = torch.masked_select(sr, mask_matrix)
                sr = sr.contiguous().view(batch_size, channels, gt_height, gt_width)
                sum_psnr += psnr_between_rgb(gt, sr)

                sr = zero_pad_to(sr, pad_image_size)
                gt = zero_pad_to(gt, pad_image_size)

                f_score += (d_model(sr)<0.5).sum().item()/batch_size
                t_score += (d_model(gt)>0.5).sum().item()/batch_size

        ###

        print('PSNR: {:.2f}, D_Fake_Acc: {:.2f}%, D_Truth_ACC: {:.2f}%'.format(sum_psnr/len(valid_dataloader), f_score*100.0/len(valid_dataloader), t_score*100.0/len(valid_dataloader)))


print("done")
torch.save(g_model.state_dict(), os.path.join(model_root, "g_model.pth"))
torch.save(d_model.state_dict(), os.path.join(model_root, "d_model.pth"))
print("saved")

# Test

In [ ]:
# For large image testing
# Divides the image and puts them back together after SR

import os
import math
import numpy as np
import torch
from PIL import Image
import torch.nn.functional as F

#################
upscale_factors = [2.22]
#################

test_root = os.path.join(drive_root, 'dataset/test')
files = os.listdir(os.path.join(test_root, 'origin'))

# model = Generator().to(device)
# model.load_state_dict(torch.load(os.path.join(drive_root, "model/dis_train1/model_50.pth"), weights_only=True))
g_model.eval()

avg = 0

i = 0
for upscale_factor in upscale_factors:
  original = upscale_factor
  upscale_factor = round(upscale_factor, 1)
  for file in files:
      # Open the high-resolution image
      Error = False
      with Image.open(os.path.join(test_root, 'origin', file)) as hr_image:

          hr_width, hr_height = hr_image.size

          if hr_width <= 256 or hr_height <= 256:
              continue

          N_w = 1 if hr_width <= 512 else max(1, math.ceil(hr_width / 512))
          N_h = 1 if hr_height <= 512 else max(1, math.ceil(hr_height / 512))

          patch_width = hr_width // N_w
          patch_height = hr_height // N_h
          width = patch_width
          height = patch_height

          sr_full = np.zeros((hr_height, hr_width, 3), dtype=np.uint8)

          total_psnr = 0
          count = 0
          Error = False

          for i in range(N_h):
              if Error:
                break

              for j in range(N_w):
                  if Error:
                    break
                  if i == 0 and j == 0 :
                      current_patch_width = patch_width
                      current_patch_height = patch_height
                  else:
                      current_patch_width = width
                      current_patch_height = height

                  left = j * current_patch_width
                  upper = i * current_patch_height
                  right = min(hr_width, left + current_patch_width)
                  lower = min(hr_height, upper + current_patch_height)

                  hr_patch = hr_image.crop((left, upper, right, lower))

                  lr_width = int(round(hr_patch.width / upscale_factor))
                  lr_height = int(round(hr_patch.height / upscale_factor))

                  adjusted_hr_width = int(round(lr_width * upscale_factor))
                  adjusted_hr_height = int(round(lr_height * upscale_factor))

                  hr_patch = hr_patch.resize((adjusted_hr_width, adjusted_hr_height), resample=Image.BICUBIC)

                  lr_patch = hr_patch.resize((lr_width, lr_height), resample=Image.BICUBIC)

                  hr = np.array(hr_patch).astype(np.float32).transpose((2, 0, 1)) / 255.
                  hr = np.expand_dims(hr, 0)
                  lr = np.array(lr_patch).astype(np.float32).transpose((2, 0, 1)) / 255.
                  lr = np.expand_dims(lr, 0)

                  hr = torch.from_numpy(hr).to(device, non_blocking=True)
                  lr = torch.from_numpy(lr).to(device, non_blocking=True)

                  batch_size, channels, lr_height_patch, lr_width_patch = lr.size()
                  _, _, hr_height_patch, hr_width_patch = hr.size()

                  pos_matrix, mask_matrix = weight_prediction_matrix_from_lr(lr_height_patch, lr_width_patch, upscale_factor)
                  pos_matrix = pos_matrix.to(device, non_blocking=True)
                  mask_matrix = mask_matrix.to(device, non_blocking=True)
                  try:
                    with torch.no_grad():
                        sr = g_model(lr, pos_matrix, upscale_factor)
                        sr = torch.masked_select(sr, mask_matrix)

                        sr = sr.contiguous().view(batch_size, channels, hr_height_patch, hr_width_patch)


                    psnr_patch = psnr_between_rgb(hr, sr)
                  except RuntimeError as e:
                      Error = True
                  if Error:
                    break
                  total_psnr += psnr_patch
                  count += 1

                  sr_np = sr.mul(255.0).cpu().numpy()
                  sr_np = np.clip(sr_np, 0.0, 255.0).astype(np.uint8)
                  sr_np = sr_np.squeeze().transpose((1, 2, 0))

                  height, width, channels = sr_np.shape


                  sr_full[upper:upper + height, left:left + width, :] = sr_np

                  # Clean up GPU memory
                  del hr, lr, sr, sr_np, pos_matrix, mask_matrix
                  torch.cuda.empty_cache()
          if Error:
            print("skipped")
            continue
          i+=1
          avg_psnr = total_psnr / count
          avg += avg_psnr
          print(f'{file}, Average PSNR: {avg_psnr:.3f}')

          original_name = os.path.splitext(file)[0]
          # Save the full SR image
          sr_image_name = f"sr_{original_name}_factor_{original}_psnr_{avg_psnr:.2f}.png"
          sr_image_full = Image.fromarray(sr_full)
          sr_image_full.save(os.path.join(test_root, 'sr', sr_image_name))

          # Save the high-resolution (HR) image with updated naming convention
          hr_image_name = f"hr_{original_name}_original.png"
          hr_image.save(os.path.join(test_root, 'hr', hr_image_name))

          # Save the low-resolution (LR) image with updated naming convention
          lr_image_name = f"lr_{original_name}_factor_{original}.png"
          lr_image_full = hr_image.resize((int(hr_width / upscale_factor), int(hr_height / upscale_factor)), resample=Image.BICUBIC)
          lr_image_full.save(os.path.join(test_root, 'lr', lr_image_name))

          torch.cuda.empty_cache()

avg /= len(files)
print(f'Average PSNR over all images: {avg:.4f}')

skipped
Average PSNR over all images: 0.0000


In [ ]:
# For small image testing

import os

#################
upscale_factors = [2,4]
#################

test_root = os.path.join(drive_root,'dataset/test')
files = os.listdir(os.path.join(test_root, 'origin'))
torch.cuda.empty_cache()
# model = Generator().to(device)
# model.load_state_dict(torch.load(os.path.join(drive_root, "model/dis_train1/model_50.pth") ,weights_only=True))
g_model.eval()



for u in upscale_factors :
    avg = 0
    for file in files:
        # if file != "ss1.png": continue
        with Image.open(os.path.join(test_root, 'origin', file)) as hr_image:
            lr_shape = (int(math.floor(hr_image.width/u)),
                            int(math.floor(hr_image.height/u)))

            hr_shape = (int(u * lr_shape[0]),
                                int(u * lr_shape[1]))

            hr_image = hr_image.crop((0, 0, hr_shape[0], hr_shape[1]))
            lr_image = hr_image.resize(lr_shape, resample=Image.BICUBIC)

            hr = np.array(asarray(hr_image)).astype(np.float32).transpose((2, 0, 1))/255.
            hr = np.expand_dims(hr, 0)

            lr = np.array(asarray(lr_image)).astype(np.float32).transpose((2, 0, 1))/255.
            lr = np.expand_dims(lr, 0)

            lr = lr[:, :3, :, :]##

            hr = torch.from_numpy(hr).to(device, non_blocking=True)
            lr = torch.from_numpy(lr).to(device, non_blocking=True)

            batch_size, channels, lr_height, lr_width = lr.size()
            _, _, hr_height, hr_width = hr.size()
            pos_matrix, mask_matrix = weight_prediction_matrix_from_lr(lr_height, lr_width, u)
            pos_matrix = pos_matrix.to(device, non_blocking=True)
            mask_matrix = mask_matrix.to(device, non_blocking=True)

            with torch.no_grad():
                sr = g_model(lr, pos_matrix, u)
                sr = torch.masked_select(sr, mask_matrix)
                sr = sr.contiguous().view(batch_size, channels, hr_height, hr_width)

            psnr = psnr_between_rgb(hr, sr)
            avg += psnr
            print(f'{file}, PSNR: {psnr:.3f}')

            # save image
            sr = sr.mul(255.0).cpu().numpy()
            sr = np.clip(sr, 0.0,255.0).astype(np.uint8)
            sr = sr.squeeze().transpose((1,2,0))
            sr_image = Image.fromarray(sr)

            sr_image.save(os.path.join(test_root, 'sr', str(u) + "_" + file))
            hr_image.save(os.path.join(test_root, 'hr', file))
            lr_image.save(os.path.join(test_root,'lr', file))
            del hr, lr, pos_matrix, mask_matrix, sr, sr_image, hr_image, lr_image, _, lr_shape, hr_shape
            torch.cuda.empty_cache()

    g_model.train()

    avg /= len(files)
    print(f'average psnr for {u}: {avg:.4f}')


0807.png, PSNR: 19.855
0811.png, PSNR: 28.675
average psnr for 2: 24.2651
0807.png, PSNR: 18.409
0811.png, PSNR: 24.519
average psnr for 4: 21.4643
